In [1]:
import numpy as np
import pandas as pd
import time
import nmslib
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', 100)
#Содержит рейтинги, отсортированные по времени
ratings = pd.read_csv('./data/ratings.csv')
#содержит теги/полки/жанры, назначенные пользователями книгам. Теги в этом файле представлены их идентификаторами. Они сортируются по _goodreads_book_id_ по возрастанию и _count_ по убыванию
book_tags = pd.read_csv("./data/book_tags.csv")
#Содержит метаданные для каждой книги (идентификаторы goodreads, авторы, название, средний рейтинг и т. д.).
books = pd.read_csv("./data/books.csv")
#Здесь каждому тегу/полке присваивается идентификатор. **tags.csv** переводит идентификаторы тегов в имена.
tags_dirty = pd.read_csv("./data/tags.csv")
#Почищенный tags.csv. Содержит около 500 наиболее популярных тегов + вручную сгруппированные оставшиеся теги.
tags = pd.read_csv("./data/tags_cleaned.xls")
#Содержит идентификаторы книг, помеченных "для чтения" каждым пользователем, в виде пар _user_id,book_id_, отсортированных по времени. Существует около миллиона пар
to_read = pd.read_csv("./data/to_read.csv")


In [2]:
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', 100)
books.head(5)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [3]:
book_tags.head(3)

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173


In [4]:
#Если мы посмотрим на датафрейм tags и на датафрейм books, то заметим, что у нас есть goodreads_book_id — от сервиса goodreads, и просто book_id — который привязан к нашей таблице. Создадим словарь, с помощью которого по book_id мы сможем находить id книги. 
mapper = dict(zip(books.goodreads_book_id,books.book_id))

In [5]:
#Теперь применим этот словарь, чтобы добавить id книги в dataframe book_tags. 
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: mapper[x])

In [6]:
book_tags.head(3)

,goodreads_book_id,tag_id,count,id
1,1,11305,37174,27
4,1,33114,12716,27
5,1,11743,9954,27


In [7]:
#Чтобы работать с моделями в библиотеке LightFm, нам нужно создать разреженные матрицы. Мы будем хранить данные в формате COO (координатный формат представления данных). Вместо хранения всех значений, которые включают нулевые значения, мы будем хранить только ненулевые значения. В COO данные представлены в виде (строка, столбец, значение).
ratings_coo = sparse.coo_matrix((ratings.rating,(ratings.user_id,ratings.book_id)))
feature_ratings  = sparse.coo_matrix(([1]*len(book_tags),(book_tags.id,book_tags.tag_id)))

Далее объявляем вспомогательные константы для обучения модели. 

In [8]:
#число потоков нашего процессора
NUM_THREADS = 8 

#число параметров вектора 
NUM_COMPONENTS = 30 

#число эпох обучения
NUM_EPOCHS = 10 

На этапе создания модели мы используем библиотеку LightFM, чтобы сделать матричное разложение (ALS) наших рейтингов книг и получить два набора векторов. 

In [9]:
start_time = time.time()
#Создаём модель
model = LightFM(learning_rate=0.05, loss='warp', no_components=NUM_COMPONENTS)
 
#Разбиваем наш датасет на обучающую и тестовую выборки
train,test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=None)

#Обучаем модель
model = model.fit(train, epochs=NUM_EPOCHS, num_threads=NUM_THREADS,item_features =feature_ratings)
#Первый набор отвечает векторному представлению пользователя, а второй набор соответствует векторному представлению книг. 
print("--- %s minutes ---" % ((time.time() - start_time)/60))


--- 59.14856697718302 minutes ---


In [10]:
start_time1 = time.time()
#Тестируем нашу модель
prec_score = precision_at_k(
                     model,
                     test,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=feature_ratings).mean()
 
recall_at_k = recall_at_k(model,
                     test,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=feature_ratings).mean()

print(recall_at_k,prec_score)

0.037625123577989 0.08164308


In [11]:
 print("--- %s minutes ---" % ((time.time() - start_time1)/60))

--- 15.826706779003143 minutes ---


In [12]:
# Достаём эбмеддинги

item_biases, item_embeddings = model.get_item_representations(features=feature_ratings)

Мы получили эмбеддинги — что делать с ними дальше? Эмбеддинги нам нужны, чтобы давать предсказание к каждой книге, а точнее искать наиболее похожие. Но как быстро найти среди 10 000 книг наиболее похожую? На помощь нам приходит метод ближайших соседей, approximate k-nn, который реализован в библиотеке nmslib.  

Вместо того, чтобы перебирать все вершины, мы можем очень быстро обходить граф.

In [13]:
#Создаём наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши книги в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [14]:
#Далее давайте напишем вспомогательную функцию, чтобы осуществлять поиск с помощью nmslib.
def nearest_books_nms(book_id, index, n=10):
    nn = index.knnQuery(item_embeddings[book_id], k=n)
    return nn

Давайте попробуем написать рекомендации к какой-нибудь книге. Например, к роману «1984» Джорджа Оруэлла.

In [15]:
books[books.original_title.str.find('1984')>=0].head(2)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
845,846,5472,5472,2966408,51,151010269,9.780151e+12,"George Orwell, Christopher Hitchens",1950.0,Animal Farm & 1984,Animal Farm / 1984,eng,4.26,116197,118761,1293,1212,3276,16511,40583,57179,https://images.gr-assets.com/books/1327959366m...,https://images.gr-assets.com/books/1327959366s...


In [16]:
# Ищем похожие книги.
nbm = nearest_books_nms(846,nms_idx)[0]

In [17]:
books[books.book_id.isin(nbm)]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
13,14,7613,7613,2207778,896,452284244,9.780452e+12,George Orwell,1945.0,Animal Farm: A Fairy Story,Animal Farm,eng,3.87,1881700,1982987,35472,66854,135147,433432,698642,648912,https://images.gr-assets.com/books/1424037542m...,https://images.gr-assets.com/books/1424037542s...
47,48,4381,4381,1272463,507,307347974,9.780307e+12,Ray Bradbury,1953.0,Fahrenheit 451,Fahrenheit 451,spa,3.97,570498,1176240,30694,28366,64289,238242,426292,419051,https://images.gr-assets.com/books/1351643740m...,https://images.gr-assets.com/books/1351643740s...
54,55,5129,5129,3204877,515,60929871,9.780061e+12,Aldous Huxley,1932.0,Brave New World,Brave New World,eng,3.97,1022601,1079135,20095,26367,60328,219895,389379,383166,https://images.gr-assets.com/books/1487389574m...,https://images.gr-assets.com/books/1487389574s...
374,375,1852,1852,3252320,1384,439227143,9.780439e+12,Jack London,1903.0,The Call of the Wild,The Call of the Wild,eng,3.83,223932,248795,6770,6366,16636,62853,90382,72558,https://images.gr-assets.com/books/1452291694m...,https://images.gr-assets.com/books/1452291694s...
845,846,5472,5472,2966408,51,151010269,9.780151e+12,"George Orwell, Christopher Hitchens",1950.0,Animal Farm & 1984,Animal Farm / 1984,eng,4.26,116197,118761,1293,1212,3276,16511,40583,57179,https://images.gr-assets.com/books/1327959366m...,https://images.gr-assets.com/books/1327959366s...
902,903,667,667,287946,460,452281253,9.780452e+12,Ayn Rand,1938.0,Anthem,Anthem,eng,3.62,95620,106766,7096,6095,10982,27984,34074,27631,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
934,935,43035,43035,2949952,1085,439236193,9.780439e+12,Jack London,1906.0,White Fang,White Fang,eng,3.94,100223,114519,3017,1823,5850,27264,42175,37407,https://images.gr-assets.com/books/1475878443m...,https://images.gr-assets.com/books/1475878443s...
4591,4592,90192,90192,1483780,636,393924769,9.780394e+12,"Nathaniel Hawthorne, Robert S. Levine",1851.0,The House of the Seven Gables,The House of the Seven Gables,eng,3.45,23797,28715,1651,1373,3640,9601,8899,5202,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
8139,8140,5481,5481,16335101,80,60898526,9.780061e+12,Aldous Huxley,1958.0,Brave New World Revisited,Brave New World Revisited,en-US,3.93,11073,12286,714,231,691,2765,4567,4032,https://images.gr-assets.com/books/1410136964m...,https://images.gr-assets.com/books/1410136964s...


Найдите рекомендации для книги Thomas Harris, The Silence of the Lambs. Какие книги присутствуют в списке рекомендаций?

In [18]:
books[books.original_title.str.find('The Silence of the Lambs')>=0].head(2)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
208,209,23807,23807,22533,187,99446782,9.780099e+12,Thomas Harris,1988.0,The Silence of the Lambs,"The Silence of the Lambs (Hannibal Lecter, #2)",eng,4.14,351107,366112,3866,10268,12845,55427,123652,163920,https://images.gr-assets.com/books/1390426249m...,https://images.gr-assets.com/books/1390426249s...


In [19]:
nbm = nearest_books_nms(209,nms_idx)[0]
books[books.book_id.isin(nbm)]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
208,209,23807,23807,22533,187,99446782,9.780099e+12,Thomas Harris,1988.0,The Silence of the Lambs,"The Silence of the Lambs (Hannibal Lecter, #2)",eng,4.14,351107,366112,3866,10268,12845,55427,123652,163920,https://images.gr-assets.com/books/1390426249m...,https://images.gr-assets.com/books/1390426249s...
430,431,28877,28877,925503,191,525945563,9.780526e+12,Thomas Harris,1981.0,Red Dragon,"Red Dragon (Hannibal Lecter, #1)",eng,4.01,194013,205433,3309,3012,7790,43235,80662,70734,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
767,768,21686,21686,1234227,134,038073186X,9.780381e+12,Dennis Lehane,2003.0,Shutter Island,Shutter Island,eng,4.07,113718,124032,6990,1636,4727,22089,49875,45705,https://images.gr-assets.com/books/1329269081m...,https://images.gr-assets.com/books/1329269081s...
1801,1802,32418,32418,2992500,132,99297701,9.780099e+12,Thomas Harris,1999.0,Hannibal,"Hannibal (Hannibal Lecter, #3)",eng,3.72,57569,63555,2098,2166,5811,17220,20844,17514,https://images.gr-assets.com/books/1327356556m...,https://images.gr-assets.com/books/1327356556s...
3405,3406,18402,18402,2164481,77,034549038X,9.780345e+12,Matthew Pearl,2003.0,The Dante Club,The Dante Club,en-US,3.38,31035,33728,2206,1822,4320,11905,10640,5041,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
4421,4422,32416,32416,46673,94,385339410,9.780385e+12,Thomas Harris,2006.0,Hannibal Rising,"Hannibal Rising (Hannibal Lecter, #4)",en-US,3.44,22767,25973,1317,1468,3733,8087,7174,5511,https://images.gr-assets.com/books/1394208690m...,https://images.gr-assets.com/books/1394208690s...
5312,5313,21727,21727,593515,46,307279952,9.780307e+12,Scott B. Smith,1993.0,A Simple Plan,A Simple Plan,NaN,3.91,18628,19650,986,478,1086,4239,7690,6157,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
6112,6113,7663,7663,2651727,57,451210638,9.780451e+12,"Jeffery Hudson, Michael Crichton",1968.0,A Case of Need,A Case of Need,en-US,3.60,14109,16047,563,291,1448,5632,5664,3012,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
6495,6496,767307,767307,1441588,49,345427637,9.780345e+12,Caleb Carr,1997.0,The Angel of Darkness,"The Angel of Darkness (Dr. Laszlo Kreizler, #2)",en-US,3.94,16343,17727,883,120,775,4246,7431,5155,https://images.gr-assets.com/books/1433066571m...,https://images.gr-assets.com/books/1433066571s...


In [20]:
import pickle
with open('item_embeddings.pickle', 'wb') as file:
   pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)